In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,66,2024-03-15,-4,110.153846,95.840000,0.537277,11.180000,21.936923,0.217892,114.752308,...,13.113846,24.960000,0.198800,112.198462,0,3,4,2,16,2024
1,66,2024-03-15,1,116.984615,97.960000,0.564985,12.670769,24.913846,0.236108,118.858462,...,12.024242,21.507576,0.165742,110.113636,0,3,4,4,24,2024
2,67,2024-03-15,1,110.696970,96.383333,0.541121,12.700000,25.842424,0.224591,114.189394,...,11.304545,24.584848,0.181652,114.712121,0,3,4,2,22,2024
3,65,2024-03-15,-2,116.890625,96.789062,0.565438,11.232813,24.495313,0.215234,120.546875,...,11.436923,25.312308,0.209723,118.660000,0,3,4,3,13,2024
4,67,2024-03-15,4,114.787879,96.033333,0.561576,10.719697,25.566667,0.174152,119.472727,...,12.589394,22.937879,0.172379,111.300000,0,3,4,2,8,2024
5,66,2024-03-15,-2,119.353846,99.847692,0.537631,10.884615,27.678462,0.211615,118.686154,...,12.855385,28.716923,0.209877,117.578462,0,3,4,2,1,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-15,MIA,DET,1.294118,3.700000,1
1,2024-03-15,PHO,CHO,1.219780,4.500000,1
2,2024-03-15,ORL,TOR,1.294118,3.700000,0
3,2024-03-15,LAC,NOP,3.350000,1.344828,0
4,2024-03-15,DEN,SAS,1.186916,5.000000,1
5,2024-03-15,ATL,UTA,1.740741,2.140000,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-15,MIA,DET,1.294118,3.700000,0
1,2024-03-15,PHO,CHO,1.219780,4.500000,1
2,2024-03-15,ORL,TOR,1.294118,3.700000,0
3,2024-03-15,LAC,NOP,3.350000,1.344828,0
4,2024-03-15,DEN,SAS,1.186916,5.000000,1
5,2024-03-15,ATL,UTA,1.740741,2.140000,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-15,MIA,DET,1.294118,3.700000,0
1,2024-03-15,PHO,CHO,1.219780,4.500000,1
2,2024-03-15,ORL,TOR,1.294118,3.700000,0
3,2024-03-15,LAC,NOP,3.350000,1.344828,0
4,2024-03-15,DEN,SAS,1.186916,5.000000,1
5,2024-03-15,ATL,UTA,1.740741,2.140000,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-15,MIA,DET,1.294118,3.700000,0
1,2024-03-15,PHO,CHO,1.219780,4.500000,1
2,2024-03-15,ORL,TOR,1.294118,3.700000,0
3,2024-03-15,LAC,NOP,3.350000,1.344828,0
4,2024-03-15,DEN,SAS,1.186916,5.000000,1
5,2024-03-15,ATL,UTA,1.740741,2.140000,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-15,MIA,DET,1.294118,3.700000,1.0
1,2024-03-15,PHO,CHO,1.219780,4.500000,1.0
2,2024-03-15,ORL,TOR,1.294118,3.700000,0.0
3,2024-03-15,LAC,NOP,3.350000,1.344828,1.0
4,2024-03-15,DEN,SAS,1.186916,5.000000,1.0
5,2024-03-15,ATL,UTA,1.740741,2.140000,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-15,MIA,DET,1.294118,3.700000,1.0
1,2024-03-15,PHO,CHO,1.219780,4.500000,1.0
2,2024-03-15,ORL,TOR,1.294118,3.700000,0.0
3,2024-03-15,LAC,NOP,3.350000,1.344828,1.0
4,2024-03-15,DEN,SAS,1.186916,5.000000,1.0
5,2024-03-15,ATL,UTA,1.740741,2.140000,0.0
